In [1]:
import datetime
import json
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier


In [2]:
events = sc.textFile('s3a://meetupevents/fix2017/*/*/*/*')

In [3]:
events.take(2)

[u'{"utc_offset":7200000,"venue":{"zip":"meetup136","country":"nl","city":"Meppel","address_1":"Tuinweg","name":"Plek van Erkenning","lon":6.188538,"lat":52.680996},"rsvp_limit":0,"venue_visibility":"public","visibility":"public","maybe_rsvp_count":0,"description":"<p>Kies voor een Reiki behandeling of een intuitieve healing. Ik help je graag op je pad naar een leven waarin je alleen maar hoeft te volgen waar je blij van wordt. Soms betekent dit eerst het loslaten van oude patronen of het genezen van (oude) wonden. Energetische healing is hierbij een stukje van de puzzel en zal je precies geven wat jij op dit moment kan gebruiken. Je kan een healing boeken op elke dag van de week. Dag of avond. Ik heb een locatie in IJhorst en Meppel. Daarnaast ben ik flexibel en kom ik ook graag bij jou langs (reiskosten in overleg).<\\/p> \\n<p>Een behandeling is \u20ac30 Inclusief kopje thee en wat lekkers. De duur van een healing is gemiddeld een uur. Je kan zelf een tijdstip uitkiezen. Trek daarna

In [4]:
def outdoors_vs_tech(event_stream):
    try:
        event = json.loads(event_stream)
        description = event.get('description')
        group_cat = event.get('group')
        if group_cat:
            group_cat = group_cat.get('category')
            

        if group_cat == 'outdoors/adventure':
            return [description, 0]
        elif group_cat == 'tech':
            return [description, 1]
        else:
            pass
    except ValueError:
        pass

In [5]:
def status_binarizer(event_stream):
    try:
        event = json.loads(event_stream)
        description = event.get('description')
        status = event.get('status')            

        if status == 'upcoming':
            return [description, 0]
        elif status == 'suggested':
            return [description, 1]
        else:
            pass
    except ValueError:
        pass

In [40]:
events_df = events.map(status_binarizer).filter(\
    lambda x: x is not None).toDF(['description', 'status']).cache()

In [8]:
# normal_tokenizer = Tokenizer(inputCol="description", outputCol="words")
tokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern='\s+|[,.\"]')
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
idf = IDF(inputCol="rawFeatures", outputCol="features")
RFC = RandomForestClassifier(labelCol="status", featuresCol="features", numTrees=10)

pipeline = Pipeline(stages=[\
                tokenizer,
                hashingTF,
                idf,
                RFC])

In [9]:
pipeline.getStages()

[RegexTokenizer_464884df1fdfcab51200,
 HashingTF_42169f9a275ed2d2d136,
 IDF_4178a2d59bcfa3756654,
 RandomForestClassifier_4e4b9e9640dd195834d4]

In [10]:
events_df_train, events_df_test = events_df.randomSplit([0.7, 0.3], seed=123)

In [13]:
events_df_train.take(1)

[Row(description=None, status=0)]

In [27]:
model = normal_tokenizer.transform(events_df_train) 

In [17]:
events_df_test.take(15)

[Row(description=None, status=0),
 Row(description=None, status=0),
 Row(description=None, status=0),
 Row(description=None, status=0),
 Row(description=None, status=0),
 Row(description=None, status=0),
 Row(description=u'<a href="https://www.facebook.com/hellskitchenbar/photos/pb.96795601960.-2207520000.1492599801./10154472259391961/?type=3"></a>\n<p><a href="https://www.facebook.com/hellskitchenbar/photos/pb.96795601960.-2207520000.1492599801./10154472259391961/?type=3">Join us for a comedy show with High Energy Tyrone Davis and other special Guests</a></p>\n<p><a href="https://www.facebook.com/hellskitchenbar/photos/pb.96795601960.-2207520000.1492599801./10154472259391961/?type=3">Event is free!<br></a></p> \n<p>Visit our page at </p> \n<p><a href="https://www.facebook.com/comedyinhell/">https://www.facebook.com/comedyinhell/</a></p>\n<p> Tyrone Davis\u2019s website: </p>\n<p><a href="http://tyroned.com/">http://tyroned.com/</a></p> \n<br>', status=0),
 Row(description=u'<a href="h